Imports

In [1]:
import os
from tqdm import tqdm
from statistics import mean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
import cmasher as cmr
import sys
from typing import List, Iterable, Optional
from functools import partial
import time
import random
import math
import torch
from torch import Tensor
from tokenizers import Tokenizer
import torch_directml
from datasets import load_dataset, Dataset
from extended_watermark_processor import WatermarkLogitsProcessor, WatermarkDetector
from transformers import (AutoTokenizer, 
                          AutoModelForSeq2SeqLM, 
                          AutoModelForCausalLM,
                          LogitsProcessorList,
                          BertTokenizer, 
                          BertForMaskedLM)

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

C:\Users\viene\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load dataset

In [2]:
dataset_name = "tatsu-lab/alpaca"

dataset = load_dataset(dataset_name, split="train", streaming=True, trust_remote_code=True)

# log an example
ds_iterator = iter(dataset)
idx = 75 # if this is c4, it's the schumacher example lol
i = 0
while i < idx: 
    next(ds_iterator)
    i += 1

example = next(ds_iterator)
print(example)

{'instruction': 'Identify the conjunctions in the following sentence', 'input': 'She wrote a letter and sealed it', 'output': 'The conjunctions in the sentence are "and".', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nIdentify the conjunctions in the following sentence\n\n### Input:\nShe wrote a letter and sealed it\n\n### Response:\nThe conjunctions in the sentence are "and".'}


Load model

In [17]:
hf_model_name = "facebook/opt-1.3b"

model = AutoModelForCausalLM.from_pretrained(hf_model_name)

tokenizer = AutoTokenizer.from_pretrained(hf_model_name)

# defaults to device 0
# will need to use 'parallelize' for multi-gpu sharding
device = torch.device("cpu")
model = model.to(device)
model.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer_norm): La

# T=1

Generate output text 

In [18]:
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,
                                               delta=2.0,
                                               seeding_scheme="selfhash") #equivalent to `ff-anchored_minhash_prf-4-True-15485863`
# Note:
# You can turn off self-hashing by setting the seeding scheme to `minhash`.

example = next(ds_iterator)

input = example["instruction"] + example["input"]

tokenized_input = tokenizer(input, return_tensors='pt')

tokenized_input = {k: v.to(device) for k, v in tokenized_input.items()}
# note that if the model is on cuda, then the input is on cuda
# and thus the watermarking rng is cuda-based.
# This is a different generator than the cpu-based rng in pytorch!

output_tokens = model.generate(**tokenized_input,
                                do_sample=True, 
                                top_k=0,
                                temperature=0.7,
                                max_new_tokens=200,
                                num_beams=1,
                                logits_processor=LogitsProcessorList([watermark_processor])
                                )

# if decoder only model, then we need to isolate the
# newly generated tokens as only those are watermarked, the input/prompt is not
output_tokens = output_tokens[:,tokenized_input["input_ids"].shape[-1]:]

output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]
print(input)
print("LLM response:\n")
print(output_text)

Create a Twitter post to promote your new product.Alisha's All Natural Jam
LLM response:

 is a jam of organic sugarcane juice, sugarcane molasses and sunflower seeds.

Use a hashtag to promote your campaign. #allnaturaljam

Write an email that you will forward to your fans. You should also include a link back to your website or social media page.

You should be able to include a short video with your campaign.

The more details you include on your social media page, the more likely your fans are to share your product on social media.

Create a Facebook post with the hashtag #allnaturaljam

Write an email about your campaign that your fans can follow and forward. You should also include a link back to your website or social media page.

You should be able to include a short video with your campaign.

The more details you include on your social media page, the more likely your fans are to share your product on social media.

Create a short Google+ post with the


Evaluate text

In [ ]:
watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
                                        gamma=0.25, # should match original setting
                                        seeding_scheme="selfhash", # should match original setting
                                        device=model.device, # must match the original rng device type
                                        tokenizer=tokenizer,
                                        z_threshold=4.0,
                                        normalizers=[],
                                        ignore_repeated_ngrams=True)

score_dict = watermark_detector.detect(output_text) # or any other text of interest to analyze

print(score_dict)

{'num_tokens_scored': 132, 'num_green_tokens': 76, 'green_fraction': 0.5757575757575758, 'z_score': 8.643325211229223, 'p_value': 2.7300111309182428e-18, 'z_score_at_T': tensor([1.7321, 2.4495, 3.0000, 3.4641, 2.8402, 2.3570, 1.9640, 2.4495, 2.8868,
        2.5560, 2.2630, 2.6667, 3.0424, 2.7775, 3.1305, 3.4641, 3.2206, 3.5382,
        3.8411, 3.6148, 3.4017, 3.6927, 3.9727, 3.7712, 4.0415, 4.3027, 4.1111,
        4.3644, 4.6101, 4.8488, 4.6663, 4.8990, 5.1257, 5.3468, 5.5626, 5.7735,
        5.6000, 5.4322, 5.6395, 5.8424, 5.6805, 5.8797, 6.0751, 5.9186, 6.1107,
        6.2993, 6.1477, 6.0000, 5.8560, 5.7155, 5.9017, 5.7646, 5.9479, 5.8140,
        5.6830, 5.8635, 5.7354, 5.9132, 5.7877, 5.6647, 5.5442, 5.7192, 5.8919,
        6.0622, 5.9438, 6.1118, 6.2776, 6.1612, 6.3249, 6.2106, 6.0982, 5.9876,
        6.1492, 6.0404, 5.9333, 5.8279, 5.9874, 6.1450, 6.3008, 6.1968, 6.0943,
        5.9932, 6.1471, 6.2994, 6.1996, 6.3502, 6.2517, 6.4008, 6.3035, 6.4510,
        6.5970, 6.7416, 6.8849

Simultate paraphrased text

In [20]:
model_name = "bert-base-uncased"
paraphrase_tokenizer = BertTokenizer.from_pretrained(model_name)
paraphrase_model = BertForMaskedLM.from_pretrained(model_name)
paraphrase_model = paraphrase_model.to(device)

def mask_tokens(tokens, mask_fraction=0.3):
    """Randomly mask some tokens"""
    n_to_mask = max(1, int(len(tokens) * mask_fraction))
    mask_indices = random.sample(range(1, len(tokens)-1), n_to_mask)  # skip [CLS] and [SEP]
    masked_tokens = tokens.copy()
    for idx in mask_indices:
        masked_tokens[idx] = paraphrase_tokenizer.mask_token
    return masked_tokens, mask_indices

def iterative_paraphrase(text, mask_fraction=0.3):
    # Tokenize
    tokens = paraphrase_tokenizer.tokenize(text)
    tokens = [paraphrase_tokenizer.cls_token] + tokens + [paraphrase_tokenizer.sep_token]
    masked_tokens, mask_indices = mask_tokens(tokens, mask_fraction)
    
    for idx in mask_indices:
        input_ids = paraphrase_tokenizer.convert_tokens_to_ids(masked_tokens)
        input_tensor = torch.tensor([input_ids]).to(device)
        
        with torch.no_grad():
            outputs = paraphrase_model(input_tensor)
            predictions = outputs.logits
        
        topk = torch.topk(predictions[0, idx], k=50)
        for token_id in topk.indices:
            token = paraphrase_tokenizer.convert_ids_to_tokens([token_id.item()])[0]
            if not token.startswith("[unused") and token != paraphrase_tokenizer.unk_token:
                predicted_token = token
                break
        else:
            predicted_token = paraphrase_tokenizer.unk_token
        
        masked_tokens[idx] = predicted_token
    
    # Detokenize
    return paraphrase_tokenizer.convert_tokens_to_string(masked_tokens[1:-1])  # remove [CLS], [SEP]

paraphrased = iterative_paraphrase(output_text, mask_fraction=0.3)

print(paraphrased)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


is a jam of organic sugarcane juice , sugarcane mochacha and sunflower oil . create a hashtag to promote your new product # allnaturaljam is an email that you send forward to your fans . you should also create a facebook back - up page or social media page . you should be able to include a short video with your campaign . the more details you include on your social media page , the more likely your fans are to share your product on social media . create a facebook post with the hashtag # allnaturalja and write some comments about your product that your fans can follow you forward . you should also include writing it back on your website or social media page . should also be able to share a short video with your campaign . the more details you include on your social media page , the more likely your fans are to share your product on social media . create a short , short video with the


Evaluate paraphrased text

In [21]:
paraphrased_score_dict = watermark_detector.detect(paraphrased) # or any other text of interest to analyze

print(paraphrased_score_dict)

{'num_tokens_scored': 147, 'num_green_tokens': 62, 'green_fraction': 0.4217687074829932, 'z_score': 4.809523809523809, 'p_value': 7.564513773304572e-07, 'z_score_at_T': tensor([1.7321, 2.4495, 3.0000, 3.4641, 2.8402, 2.3570, 1.9640, 2.4495, 2.8868,
        2.5560, 2.2630, 2.6667, 2.4019, 2.7775, 2.5342, 2.3094, 2.1004, 1.9052,
        1.7219, 2.0656, 2.3938, 2.7080, 2.5281, 2.8284, 2.6558, 2.4910, 2.7778,
        3.0551, 3.3235, 3.1623, 3.4219, 3.2660, 3.1156, 2.9704, 2.8301, 2.6943,
        2.9424, 2.8098, 3.0509, 3.2863, 3.1558, 3.3853, 3.6098, 3.4816, 3.3566,
        3.2348, 3.1160, 3.0000, 2.8868, 2.7761, 2.9913, 2.8823, 2.7757, 2.6713,
        2.8804, 2.7775, 2.6765, 2.8808, 3.0817, 2.9814, 3.1787, 3.3729, 3.2733,
        3.4641, 3.3657, 3.2691, 3.1741, 3.3607, 3.2667, 3.1743, 3.0833, 2.9938,
        2.9057, 2.8189, 2.7333, 2.9140, 3.0924, 3.0071, 3.1829, 3.0984, 3.0151,
        2.9329, 3.1052, 3.2757, 3.4442, 3.6109, 3.7758, 3.9389, 4.1003, 4.0166,
        4.1761, 4.3339, 4.4901,

# T=t

Generate output text 

Evaluate text

Simultate paraphrased text

Evaluate paraphrased text